In [49]:
import pandas as pd

In [50]:
xl = pd.ExcelFile('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx')

xl.sheet_names  # see all sheet names

['percent GDP', 'GDP']

In [51]:
GDP = xl.parse("GDP")  # read a specific sheet to DataFrame

In [52]:
GDP2 = GDP.set_index(["wbregionname","year","ssp"]).gdp.unstack("wbregionname")

In [53]:
GDP2.columns

Index(['East Asia and Pacific', 'Europe and Central Asia',
       'Latin America and Caribbean', 'Middle East and North Africa',
       'South Asia', 'Sub-Saharan Africa'],
      dtype='object', name='wbregionname')

In [54]:
GDP2 = GDP2.rename(columns={"Europe and Central Asia":"Eastern Europe and Central Asia",
                            "Middle East and North Africa":"Middle-East and North Africa"})

In [55]:
GDP2["Africa"] = GDP2["Middle-East and North Africa"]+GDP2["Sub-Saharan Africa"]
GDP2["Asia"] = GDP2["East Asia and Pacific"]+GDP2["South Asia"]
GDP2["all_countries"] = GDP2["Africa"]+GDP2["Asia"]+GDP2["Latin America and Caribbean"]+GDP2["Eastern Europe and Central Asia"]

In [56]:
GDP2.stack().sample(5)

year  ssp   wbregionname                   
2022  ssp2  all_countries                      4.781525e+13
2021  ssp5  Asia                               2.611930e+13
2029  ssp5  South Asia                         9.726712e+12
2024  ssp4  Eastern Europe and Central Asia    4.515983e+12
2028  ssp5  all_countries                      7.225835e+13
dtype: float64

In [57]:
res = xl.parse("percent GDP")

In [58]:
res["sector"] = res.sector.replace({"WASH":"wash"})

In [59]:
res.region.unique()

array(['all_countries', 'Africa', 'Asia', 'Latin America and Caribbean',
       'Former soviet Union', 'Middle-East and North Africa',
       'Sub-Saharan Africa', 'South Asia', 'East Asia and Pacific',
       'Eastern Europe and Central Asia'], dtype=object)

In [60]:
res['wbregion'] = res.region.copy()
res.loc[res.region=='Africa','wbregion'] = 'Sub-Saharan Africa'
res.loc[res.region=='Asia','wbregion'] = 'East Asia and Pacific'

res.loc[res.region=='Former soviet Union','wbregion'] = 'Eastern Europe and Central Asia'

disag_afr = res.loc[res.region=='Africa'].copy()
disag_afr.loc[disag_afr.region=='Africa','wbregion'] = 'Middle-East and North Africa'

disag_asia = res.loc[res.region=='Asia'].copy()
disag_asia.loc[disag_asia.region=='Asia','wbregion'] = 'South Asia'

res = res.append(disag_afr).append(disag_asia)

In [61]:
res.loc[res.sector=="energy","ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="transport")&(res.scenario=="max"),"ssp"] = "ssp3"
res.loc[(res.sector=="wash")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="max"),"ssp"] = "ssp4"
res.loc[(res.sector=="coastal protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="max"),"ssp"] = "ssp5"
res.loc[(res.sector=="river protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="river protection")&(res.scenario=="min"),"ssp"] = "ssp3"
res.loc[(res.sector=="river protection")&(res.scenario=="max"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="irrigation")&(res.scenario=="max"),"ssp"] = "ssp2"

res.loc[(res.region=="all_countries"),"ssp"] = "ssp2"

In [62]:
res[res.ssp.isnull()]

,region,sector,scenario,urban vs rural,variable,capital vs maintenance,data,wbregion,ssp


In [63]:
mydata = res.merge(GDP2.stack().reset_index().rename(columns={"wbregionname":"wbregion",0:"GDP"}),on=["wbregion","ssp"])

In [64]:
mydata["cost_bUSD"] = 1e-9/100*mydata.data*mydata.GDP

In [65]:
mydata[mydata.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance","year"]).cost_bUSD.sum(level=["scenario","capital vs maintenance"])

scenario   capital vs maintenance
min        capital                   23196.929276
max        capital                   93025.228326
preferred  capital                   18209.193630
min        maintenance               18161.691385
max        maintenance               40297.737207
preferred  maintenance                8708.744779
best       maintenance               21534.350727
           capital                   35626.683189
maxmax     capital                   19000.897701
minmin     capital                     316.681628
Name: cost_bUSD, dtype: float64

In [66]:
d = 0.06
mydata["discount_factor"] = (1+d)**(1-(mydata.year.astype(int)-2015))

In [67]:
mydata["cost_bUSD_disc"] = mydata["cost_bUSD"]*mydata["discount_factor"]

In [68]:
newdata = mydata.set_index(["wbregion","sector","scenario","capital vs maintenance","year"]).cost_bUSD_disc.sum(level=["wbregion","sector","scenario","capital vs maintenance"])

In [69]:
newdata2 = newdata.reset_index()

In [70]:
newdata2["cost_bUSD_disc_annual"] = newdata2["cost_bUSD_disc"]/15

In [71]:
final_data = newdata2.merge(res,on=['wbregion','sector','scenario','capital vs maintenance']).drop('cost_bUSD_disc',axis=1).rename(columns={'data':'percent_GDP'})

In [78]:
urban_data = newdata2.merge(res,on=['wbregion','sector','scenario','capital vs maintenance']).drop('cost_bUSD_disc',axis=1).rename(columns={'data':'percent_GDP'})

In [81]:
urban_data = urban_data[urban_data["urban vs rural"]=="urban"].drop(["variable","ssp","region"],axis=1)

In [83]:
urban_data.to_csv("C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\urban_data.csv")

In [72]:
final_data = final_data[final_data.wbregion!="all_countries"].drop(["variable","ssp","region"],axis=1)

In [73]:
final_data2 = final_data.set_index(['wbregion','sector','scenario','capital vs maintenance']).stack()

In [74]:
final_data2 = final_data2.reset_index().rename(columns={"level_4":"usd vs percent",0:"data"})

In [75]:
final_data3 = final_data2.set_index(['wbregion','sector','scenario','capital vs maintenance',"usd vs percent"]).data.unstack("sector")

In [76]:
final_data3.columns

Index(['coastal protection', 'energy', 'irrigation', 'river protection',
       'transport', 'wash'],
      dtype='object', name='sector')

In [77]:
final_data3['flood protection'] = final_data3['coastal protection']+final_data3['river protection']

final_data3['Total'] = final_data3[['flood protection','energy','transport','irrigation','wash']].sum(axis=1)

In [30]:
final_data4 = final_data3.drop(['coastal protection','river protection'],axis=1).stack().to_frame().reset_index().rename(columns={0:"data"})

In [31]:
sector_dic = {'energy':'Energy','irrigation':'Irrigation','wash':"Water supply and sanitation","transport":"Transport","flood protection":"Flood protection"}

In [32]:
var_dic = {"percent_GDP":"Percent of GDP","cost_bUSD_disc_annual":"Billion USD"}

In [33]:
scenario_dic = {'best':'Preferred','min':'Low spending','max':'High spending'}

In [34]:
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='min'),'policies and tech'] = "High energy efficiency \
and demand management"
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='max'),'policies and tech'] = "No investment in energy \
efficiency; fossil energy for 10 years followed by switch to low carbon"
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='best'),'policies and tech'] = "Invest now in \
renewable energy; energy efficiency"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='min'),'policies and tech'] = "Decentralized systems \
(like sceptic tanks) whenever possible"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='max'),'policies and tech'] = "Sewerage systems with \
treatment"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='best'),'policies and tech'] = "Sewerage systems with \
treatment only in high population density areas"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='min'),'policies and tech'] = "Increase the utilization \
rate of rail and public transport; densify cities; reduce demand for transport through gasoline taxes"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='max'),'policies and tech'] = "Let cities sprawl; \
favor rail investments without accompanying policies"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='best'),'policies and tech'] = "Increase the \
utilization rate of rail and public transport; densify cities; promote electric mobility"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='min'),'policies and tech'] = "Subsidize irrigation \
infrastructure ;promote low-meat diets"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='max'),'policies and tech'] = "Subsidize both \
irrigation infrastructure and electricity for water extraction"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='best'),'policies and tech'] = "Subsidize irrigation \
infrastructure"
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='min'),'policies and tech'] = 'Maintain \
construction costs low through better procurement'
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='max'),'policies and tech'] = "Let construction \
costs increase"
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='best'),'policies and tech'] = 'Maintain \
construction costs low through better procurement'

In [35]:
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='min'),'goals'] = "Universal access to low \
service tiers"
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='max'),'goals'] = "Universal access to high \
service tiers, low carbon electricity"
final_data4.loc[(final_data4.sector=='energy')&(final_data4.scenario=='best'),'goals'] = "Universal access to medium \
service tiers, low carbon electricity"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='min'),'goals'] = "Universal access to basic water \
and sanitation"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='max'),'goals'] = "Universal access to safe water \
and sanitation"
final_data4.loc[(final_data4.sector=='wash')&(final_data4.scenario=='best'),'goals'] = "Universal access to safe water \
and sanitation"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='min'),'goals'] = "Low carbon transport"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='max'),'goals'] = "Follow transport demand reactively"
final_data4.loc[(final_data4.sector=='transport')&(final_data4.scenario=='best'),'goals'] = "Low carbon transport"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='min'),'goals'] = "End hunger and protect biodiversity"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='max'),'goals'] = "End hunger"
final_data4.loc[(final_data4.sector=='irrigation')&(final_data4.scenario=='best'),'goals'] = "End hunger and protect biodiversity"
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='min'),'goals'] = "Keep coastal flood risk \
constant in relative terms; invest in river protection based on cost-benefit analysis"
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='max'),'goals'] = "Invest in coastal \
and river protection based on cost-benefit analysis"
final_data4.loc[(final_data4.sector=='flood protection')&(final_data4.scenario=='best'),'goals'] = "Adopt Dutch standards \
of coastal flood protection for cities; keep river flood risk constant in absolute terms"

In [36]:
final_data4['sector'] = final_data4.sector.replace(sector_dic)
final_data4['usd vs percent'] = final_data4['usd vs percent'].replace(var_dic)
final_data4['scenario'] = final_data4.scenario.replace(scenario_dic)
final_data4['capital vs maintenance'] = final_data4['capital vs maintenance'].str.capitalize()

#final_data4['goals'] = 'Goals: '+final_data4['goals']
#final_data4['policies and tech'] = 'Policies/technologies: '+final_data4['policies and tech']

In [37]:
final_data4.sample(5)

,wbregion,scenario,capital vs maintenance,usd vs percent,sector,data,policies and tech,goals
229,South Asia,Preferred,Maintenance,Billion USD,Flood protection,2.968577,Maintain construction costs low through better...,Adopt Dutch standards of coastal flood protect...
6,East Asia and Pacific,Preferred,Capital,Percent of GDP,Irrigation,0.130000,Subsidize irrigation infrastructure,End hunger and protect biodiversity
162,Middle-East and North Africa,Preferred,Capital,Billion USD,Energy,26.381556,Invest now in renewable energy; energy efficiency,"Universal access to medium service tiers, low ..."
234,South Asia,High spending,Capital,Billion USD,Energy,153.941931,No investment in energy efficiency; fossil ene...,"Universal access to high service tiers, low ca..."
295,Sub-Saharan Africa,High spending,Capital,Percent of GDP,Transport,6.110000,Let cities sprawl; favor rail investments with...,Follow transport demand reactively


In [38]:
final_data4.to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\final_for_website.csv')

In [39]:
datawrapper = final_data4.set_index(['wbregion','scenario','capital vs maintenance','sector','usd vs percent','goals','policies and tech']).data.unstack('usd vs percent').reset_index()

In [40]:
datawrapper.sample(3)

usd vs percent,wbregion,scenario,capital vs maintenance,sector,goals,policies and tech,Billion USD,Percent of GDP
134,South Asia,Preferred,Maintenance,Water supply and sanitation,Universal access to safe water and sanitation,Sewerage systems with treatment only in high p...,12.722474,0.30
3,East Asia and Pacific,High spending,Capital,Transport,Follow transport demand reactively,Let cities sprawl; favor rail investments with...,336.603086,2.25
72,Latin America and Caribbean,Preferred,Capital,Energy,"Universal access to medium service tiers, low ...",Invest now in renewable energy; energy efficiency,91.500000,1.20


In [41]:
datawrapper.to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\datawrapper.csv')

In [47]:
datawrapper[datawrapper.wbregion=="Middle-East and North Africa"].set_index(["sector","capital vs maintenance","scenario"])[["Billion USD","Percent of GDP"]].sum(level=["capital vs maintenance","scenario"])

,usd vs percent,Billion USD,Percent of GDP
capital vs maintenance,scenario,,
Capital,High spending,219.600818,9.7610
Maintenance,High spending,43.931058,1.9530
Capital,Low spending,60.826098,2.6901
Maintenance,Low spending,27.847072,1.2302
Capital,Preferred,124.917795,5.5400
Maintenance,Preferred,35.062666,1.5550


In [48]:
datawrapper[datawrapper.wbregion=="Middle-East and North Africa"].set_index(["sector","capital vs maintenance","scenario"])[["Billion USD","Percent of GDP"]].sum(level=["capital vs maintenance","scenario","sector"])

usd vs percent                                                    Billion USD  \
capital vs maintenance scenario      sector                                     
Capital                High spending Energy                         46.224094   
                                     Flood protection                9.328809   
                                     Irrigation                      3.607734   
                                     Transport                     137.313693   
                                     Water supply and sanitation    23.126488   
Maintenance            High spending Energy                          5.411601   
                                     Flood protection                1.219491   
                                     Transport                      30.564095   
                                     Water supply and sanitation     6.735870   
Capital                Low spending  Energy                         16.685770   
                                     Flood protection                1.126525   
                                     Irrigation                      1.814057   
                                     Transport                      28.798159   
                                     Water supply and sanitation    12.401586   
Maintenance            Low spending  Energy                          4.960634   
                                     Flood protection                0.229829   
                                     Transport                      19.274358   
                                     Water supply and sanitation     3.382251   
Capital                Preferred     Energy                         26.381556   
                                     Flood protection                3.833218   
                                     Irrigation                      2.254834   
                                     Transport                      72.154683   
                                     Water supply and sanitation    20.293505   
Maintenance            Preferred     Energy                          5.862568   
                                     Flood protection                0.789192   
                                     Transport                      21.646405   
                                     Water supply and sanitation     6.764502   

usd vs percent                                                    Percent of GDP  
capital vs maintenance scenario      sector                                       
Capital                High spending Energy                               2.0500  
                                     Flood protection                     0.4110  
                                     Irrigation                           0.1600  
                                     Transport                            6.1100  
                                     Water supply and sanitation          1.0300  
Maintenance            High spending Energy                               0.2400  
                                     Flood protection                     0.0530  
                                     Transport                            1.3600  
                                     Water supply and sanitation          0.3000  
Capital                Low spending  Energy                               0.7400  
                                     Flood protection                     0.0501  
                                     Irrigation                           0.0800  
                                     Transport                            1.2700  
                                     Water supply and sanitation          0.5500  
Maintenance            Low spending  Energy                               0.2200  
                                     Flood protection                     0.0102  
                                     Transport                            0.8500  
                                     Water supply and sanitation         

In [42]:
final_data.sort_values(["wbregion","sector","scenario"]).to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\semi_final_for_website.csv')

In [43]:
totals = pd.DataFrame()

# Energy

In [44]:
energy = newdata2.loc[(newdata2.sector=="energy"),:]

## keep for overview

In [45]:
energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()

AttributeError: 'DataFrame' object has no attribute 'region'

In [ ]:
temp = energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "energy"

In [ ]:
totals = temp

In [ ]:
energy.loc[energy.region=="all_countries",["scenario","cost_bUSD_disc_annual","capital vs maintenance"]]

# Transport

In [ ]:
transport = newdata2.loc[(newdata2.sector=="transport"),:]

## keep for overview

In [ ]:
transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
temp = transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "transport"

In [ ]:
totals = totals.append(temp)

In [ ]:
transport.loc[transport.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

# WASH

In [ ]:
wash = newdata2.loc[(newdata2.sector=="wash"),:]

## keep for overview

In [ ]:
wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
temp = wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "wash"

In [ ]:
totals = totals.append(temp)

In [ ]:
wash.loc[wash.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

# Flood

In [ ]:
coastal_flood = newdata2.loc[(newdata2.sector=="coastal protection"),:]

In [ ]:
coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
river_flood = newdata2.loc[(newdata2.sector=="river protection"),:]

In [ ]:
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
river_flood.loc[river_flood.region=="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
flood = coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])+\
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

## keep for overview

In [ ]:
flood

In [ ]:
temp = flood.reset_index()
temp["sector"] = "flood"

In [ ]:
totals = totals.append(temp)

In [ ]:
newdata2.loc[(newdata2.sector=="flood"),:]

# Irrigation

In [ ]:
irrigation = newdata2.loc[(newdata2.sector=="irrigation"),:]

## keep for overview

In [ ]:
irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
temp = irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "irrigation"

In [ ]:
totals = totals.append(temp)

In [ ]:
irrigation.loc[irrigation.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

In [ ]:
newdata2[(newdata2.region!="all_countries")].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

In [ ]:
newdata2[(newdata2.region=="all_countries")&(newdata2["capital vs maintenance"]=="capital")].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

## we don't have energy maintenance per region so we use global %

In [ ]:
temp2 = energy.loc[(energy.region=="all_countries")&(energy["capital vs maintenance"]=="maintenance")&(energy.scenario!="best"),["scenario","cost_bUSD_disc_annual","capital vs maintenance"]]
temp2["sector"] = "energy"

In [ ]:
temp2

In [ ]:
totals = totals.append(temp2)

In [ ]:
totals[totals.sector=="energy"]

In [ ]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance")

In [ ]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").sum(level="scenario")

totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_usd_results.csv')

In [ ]:
res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance")

res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_percent_results.csv')

In [ ]:
newdata2[newdata2.sector=="irrigation"].sort_values("region")

In [ ]:
region_totals = newdata2.set_index(["region","sector","scenario","capital vs maintenance"]).drop("all_countries",level="region").sum(level=["region","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").capital

In [ ]:
region_totals = region_totals.reset_index()

In [ ]:
region_totals['region2'] = region_totals.region

region_totals.loc[region_totals.region=='Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Middle-East and North Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Asia','region2']='Asia'
region_totals.loc[region_totals.region=='South Asia','region2']='Asia'
region_totals.loc[region_totals.region=='East Asia and Pacific','region2']='Asia'

In [ ]:
region_totals2 = region_totals.set_index(['region2','scenario']).capital.sum(level=["region2",'scenario'])

In [ ]:
region_totals2 = region_totals2.unstack("scenario")

In [ ]:
region_totals2.sum(axis=0)

In [ ]:
region_totalspc = region_totals2/region_totals2.sum(axis=0)

In [ ]:
region_totalspc

faux: ne pas faire le truc ci-dessous (a corriger)

In [ ]:
region_pc_totals = mydata.copy()
region_pc_totals['region2'] = region_pc_totals.region

region_pc_totals.loc[region_pc_totals.region=='Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Middle-East and North Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='South Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='East Asia and Pacific','region2']='Asia'

In [ ]:
region_pc_totals.columns

In [ ]:
region_pc_totals.sample(5)

In [ ]:
region_pc_totals = region_pc_totals.set_index(['sector','scenario','capital vs maintenance','region2','year'])[['cost_bUSD','cost_bUSD_disc','GDP']].sum(level=['scenario','sector','capital vs maintenance','region2','year']).drop("all_countries",level="region2")

In [ ]:
region_pc_totals.sample(10)

In [ ]:
region_pc_totals2 = region_pc_totals[['cost_bUSD','cost_bUSD_disc']].sum(level=['scenario','capital vs maintenance','region2','year'])

In [ ]:
region_pc_totals2['GDP'] = region_pc_totals.GDP.mean(level=['scenario','capital vs maintenance','region2','year'])

In [ ]:
region_pc_totals2.sample(3)

In [ ]:
region_pc_totals2['new_data'] = 100*1e9*region_pc_totals2.cost_bUSD/region_pc_totals2.GDP

In [ ]:
region_pc_totals_mean = region_pc_totals2['new_data'].mean(level=['scenario','capital vs maintenance','region2']).to_frame()

In [ ]:
region_pc_totals_mean['cost_bUSD_disc'] = 1/15*region_pc_totals2['cost_bUSD_disc'].sum(level=['scenario','capital vs maintenance','region2'])

In [ ]:
region_pc_totals_mean.unstack("capital vs maintenance")

region_pc_totals_mean.unstack("capital vs maintenance").to_excel("C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\aggregated_regions_numbers.xlsx")

In [ ]:
region_pc_totals3 = 1/15*region_pc_totals[['cost_bUSD_disc']].sum(level=['scenario','capital vs maintenance','region2','sector'])

In [ ]:
region_pc_totals3 = region_pc_totals3.reset_index()

In [ ]:
region_pc_totals3[(region_pc_totals3.sector=="energy")&(region_pc_totals3["capital vs maintenance"]=="maintenance")]